In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')


In [ ]:
%matplotlib  inline

In [ ]:
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pyarrow as pa
import pyarrow.parquet as pq


### Mise au format du fichier des maillots jaunes

In [ ]:
with open('vainqueurs.txt')  as file:
    lignes = file.readlines()
    

In [ ]:
re_annee = r'(\d{4})'
re_vainqueur = r'1\. \w\. (.+?)2'
listev = []
for ligne in lignes: 
   # print(ligne)
    tan = re.match(re_annee, ligne)
    if tan:
        annee = tan[1]
        tnom = re.search(re_vainqueur, ligne)
        if tnom:
            if '('  in tnom[1]:
                nom = tnom[1][:-6]
            else:
                nom = tnom[1]
        else:
            nom =None
        listev.append([annee, nom])        
listev   


In [ ]:
datav = pd.DataFrame(listev, columns =['annee', 'nom'])

In [ ]:
datav.head()

### Regrouper par nom

In [ ]:
gvain = datav.groupby('nom')

In [ ]:
gvain['nom'].count().nlargest(5)

In [ ]:
comp2 = gvain['nom'].count()

In [ ]:
print(comp2.loc['Anquetil'])

### Charger le fichier parquet des vainqueurs d'étapes

In [ ]:
data1= pd.read_parquet('vetape.parquet')
data1.info()

In [ ]:
comp2.loc['Thys']

In [ ]:
re_nom = r' ([A-Z]\w+)'
lv = []
for item in data1['Winner'] :
    print(item)
    nom = re.search(re_nom, item)
    if nom:
       # print(nom[1])
       try:
           cherche = comp2.loc[nom[1]]
           lv.append(True)
       except:
           lv.append(False) 
print(lv)            

In [ ]:
data1['vainqueur'] = lv

### Visualisation

In [ ]:
from matplotlib import patches as mpatches
handles = [mpatches.Patch(color='g') , mpatches.Patch(color='y')]
labels = ['maillot vert', 'maillot jaune']
           
colors = np.array(['g']*len(lv))
colors[lv] ='y'
plt.figure(figsize =(15,10))
plt.bar(data1['Winner'],data1['Stage'],color = colors, )
plt.legend(handles, labels)     
plt.xticks(rotation='vertical')
plt.xlabel('nom')
plt.ylabel('nombre d etape')
plt.title('hall de la légende')
plt.savefig('mongraphvainqueur.png')